In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

In [2]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(1)

In [3]:
df.head(2).to_dict()

{'Pres': {0: 957.9, 1: 958.0},
 'Pres_max': {0: 957.9, 1: 958.3},
 'Pres_min': {0: 957.1, 1: 957.9},
 'Radiacao': {0: 0.0, 1: 0.0},
 'Temp': {0: 21.7, 1: 21.4},
 'Temp_orvalho': {0: 21.7, 1: 21.4},
 'Temp_max': {0: 23.7, 1: 21.7},
 'Temp_min': {0: 21.7, 1: 21.4},
 'Temp_orvalho_max': {0: 23.7, 1: 21.6},
 'Temp_orvalho_min': {0: 21.7, 1: 21.4},
 'Umid_max': {0: 100.0, 1: 100.0},
 'Umid_min': {0: 100.0, 1: 100.0},
 'Umid': {0: 100.0, 1: 100.0},
 'Dir_vento': {0: 130.5, 1: 109.0},
 'Rajada_vento': {0: 3.75, 1: 5.4},
 'Vel_vento': {0: 2.1, 1: 3.35},
 'Latitude': {0: -21.927251, 1: -21.927251},
 'Longitude': {0: -50.490251, 1: -50.490251},
 'Altitude': {0: 498.0, 1: 498.0},
 'datahora': {0: Timestamp('2021-01-01 00:00:00+0000', tz='UTC'),
  1: Timestamp('2021-01-01 01:00:00+0000', tz='UTC')},
 'classe_chuva': {0: 0, 1: 0}}

In [4]:
df.isnull().sum()

Pres                0
Pres_max            0
Pres_min            0
Radiacao            0
Temp                0
Temp_orvalho        0
Temp_max            0
Temp_min            0
Temp_orvalho_max    0
Temp_orvalho_min    0
Umid_max            0
Umid_min            0
Umid                0
Dir_vento           0
Rajada_vento        0
Vel_vento           0
Latitude            0
Longitude           0
Altitude            0
datahora            0
classe_chuva        0
dtype: int64

In [5]:
def subsample_sequence(df, length):
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [6]:
df_sample = subsample_sequence(df,48)
df_sample.head()

,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,Temp_orvalho_min,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
1334,955.7,956.0,955.7,3135.0,27.7,22.9,28.3,26.5,23.7,22.3,...,74.0,75.0,166.0,4.25,2.00,-21.927251,-50.490251,498.0,2021-02-25 14:00:00+00:00,0
1335,955.4,955.8,955.4,3587.8,29.0,24.4,29.3,27.6,24.4,22.6,...,68.0,76.0,123.5,3.00,1.55,-21.927251,-50.490251,498.0,2021-02-25 15:00:00+00:00,0
1336,954.6,955.4,954.6,3484.4,29.7,22.8,30.6,28.9,24.5,22.8,...,65.0,67.0,202.0,7.10,2.90,-21.927251,-50.490251,498.0,2021-02-25 16:00:00+00:00,0
1337,953.6,954.6,953.6,3352.4,32.0,23.3,32.3,29.5,24.8,22.4,...,57.0,60.0,135.0,6.45,2.90,-21.927251,-50.490251,498.0,2021-02-25 17:00:00+00:00,0
1338,951.8,953.6,951.8,2525.1,31.4,22.5,32.4,30.7,23.9,21.7,...,55.0,59.0,173.5,9.10,4.40,-21.927251,-50.490251,498.0,2021-02-25 18:00:00+00:00,0


In [7]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['classe_chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [8]:
X,y = split_subsample_sequence(df,48)
X.shape

(24, 19)

In [46]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [10]:
X,y 

(array([[ 9.6560000e+02,  9.6570000e+02,  9.6520000e+02,  1.5695000e+03,
          1.9400000e+01,  1.2400000e+01,  1.9400000e+01,  1.5700000e+01,
          1.2500000e+01,  1.1800000e+01,  7.8000000e+01,  6.3000000e+01,
          6.4000000e+01,  1.1800000e+02,  7.6000000e+00,  4.6000000e+00,
         -2.1927251e+01, -5.0490251e+01,  4.9800000e+02],
        [ 9.6530000e+02,  9.6570000e+02,  9.6530000e+02,  2.0360000e+03,
          2.1800000e+01,  1.2200000e+01,  2.1900000e+01,  1.9300000e+01,
          1.2800000e+01,  1.1100000e+01,  6.4000000e+01,  5.1000000e+01,
          5.4000000e+01,  1.0000000e+02,  7.5000000e+00,  4.5000000e+00,
         -2.1927251e+01, -5.0490251e+01,  4.9800000e+02],
        [ 9.6480000e+02,  9.6540000e+02,  9.6480000e+02,  2.4155000e+03,
          2.3700000e+01,  1.1300000e+01,  2.3700000e+01,  2.1700000e+01,
          1.1900000e+01,  1.0800000e+01,  5.4000000e+01,  4.5000000e+01,
          4.6000000e+01,  9.1000000e+01,  7.3000000e+00,  4.2000000e+00,
        

In [11]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [12]:
df.shape[0]*0.8

7008.0

In [13]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [49]:
df_train.value_counts(['classe_chuva'])

classe_chuva
0               6580
1                391
2                 34
3                  3
dtype: int64

In [15]:
df_test.shape

(1752, 21)

In [50]:
X_train,y_train = get_X_y(df_train,6000,72)

In [51]:
X_test,y_test = get_X_y(df_test,6000,72)

In [52]:
y_train.shape

(6000, 24)

In [53]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# label = LabelEncoder()
# y_enc = label.fit_transform(y.reshape((-1,)))
y_cat = to_categorical(y_train)
y_cat.shape

(6000, 24, 4)

In [54]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

In [55]:
model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(24, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, None, 19)         39        
 ion)                                                            
                                                                 
 lstm_6 (LSTM)               (None, 20)                3200      
                                                                 
 dense_12 (Dense)            (None, 10)                210       
                                                                 
 dense_13 (Dense)            (None, 24)                264       
                                                                 
Total params: 3,713
Trainable params: 3,674
Non-trainable params: 39
_________________________________________________________________


In [56]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
150/150 [==============================] - 2s 9ms/step - loss: 0.4031 - accuracy: 0.0144 - val_loss: 0.2787 - val_accuracy: 0.0158
Epoch 2/10
150/150 [==============================] - 1s 7ms/step - loss: 0.2394 - accuracy: 0.0104 - val_loss: 0.2187 - val_accuracy: 0.0150
Epoch 3/10
150/150 [==============================] - 1s 7ms/step - loss: 0.1990 - accuracy: 0.0096 - val_loss: 0.1990 - val_accuracy: 0.0125
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 0.1887 - accuracy: 0.0146 - val_loss: 0.1881 - val_accuracy: 0.0167
Epoch 5/10
150/150 [==============================] - 1s 8ms/step - loss: 0.1833 - accuracy: 0.0148 - val_loss: 0.1849 - val_accuracy: 0.0200
Epoch 6/10
150/150 [==============================] - 1s 7ms/step - loss: 0.1774 - accuracy: 0.0160 - val_loss: 0.1786 - val_accuracy: 0.0150
Epoch 7/10
150/150 [==============================] - 1s 7ms/step - loss: 0.1716 - accuracy: 0.0194 - val_loss: 0.1757 - val_accuracy: 0.0225
Epoch 

In [57]:
model.predict(X_test)[0]

array([0.18924662, 0.14047703, 0.14976066, 0.13902634, 0.1015662 ,
       0.11022925, 0.13106924, 0.12687644, 0.07358918, 0.09618524,
       0.12700406, 0.07777372, 0.09970671, 0.10007277, 0.06561902,
       0.08878124, 0.12322101, 0.0796552 , 0.11537322, 0.13690105,
       0.13954476, 0.17154408, 0.18975788, 0.14096951], dtype=float32)